In [1]:
import os
import sys
import math
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import audioread


from torchlibrosa.stft import Spectrogram, LogmelFilterBank # use 'magphase' to extract phase informatinon with magnitude

In [2]:
pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |################################| 1982.2 MB 12 kB/s               
     |################################| 17.6 MB 31.2 MB/s            
     |################################| 1.9 MB 12.3 MB/s            
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.0
    Uninstalling torchvision-0.9.0:
      Successfully uninstalled torchvision-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
torch.__version__

'1.8.0+cu111'

In [3]:
import torch
from torch import nn
m = nn.Conv1d(16, 33, 3, stride=2)
m=m.to('cuda')
input = torch.randn(20, 16, 50)
input=input.to('cuda')
output = m(input)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:

# initialize and conv block
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


def init_gru(rnn):
    """Initialize a GRU layer. """
    
    def _concat_init(tensor, init_funcs):
        (length, fan_out) = tensor.shape
        fan_in = length // len(init_funcs)
    
        for (i, init_func) in enumerate(init_funcs):
            init_func(tensor[i * fan_in : (i + 1) * fan_in, :])
        
    def _inner_uniform(tensor):
        fan_in = nn.init._calculate_correct_fan(tensor, 'fan_in')
        nn.init.uniform_(tensor, -math.sqrt(3 / fan_in), math.sqrt(3 / fan_in))
    
    for i in range(rnn.num_layers):
        _concat_init(
            getattr(rnn, 'weight_ih_l{}'.format(i)),
            [_inner_uniform, _inner_uniform, _inner_uniform]
        )
        torch.nn.init.constant_(getattr(rnn, 'bias_ih_l{}'.format(i)), 0)

        _concat_init(
            getattr(rnn, 'weight_hh_l{}'.format(i)),
            [_inner_uniform, _inner_uniform, nn.init.orthogonal_]
        )
        torch.nn.init.constant_(getattr(rnn, 'bias_hh_l{}'.format(i)), 0)
    
def init_ln(module):
    if isinstance(module, nn.Embedding):
        module.weight.data.normal_(mean=0.0, std=1.0)
        if module.padding_idx is not None:
            module.weight.data[module.padding_idx].zero_()
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)


In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, momentum):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels, momentum)
        self.bn2 = nn.BatchNorm2d(out_channels, momentum)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        """
        Args:
          input: (batch_size, in_channels, time_steps, freq_bins)
        Outputs:
          output: (batch_size, out_channels, classes_num)
        """

        x = F.relu_(self.bn1(self.conv1(input)))
        #print(x)
        x = F.relu_(self.bn2(self.conv2(x)))
        
        if pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        
        return x

In [32]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
class TransformerEncoder(nn.Module):
    def __init__(self, d_model=768, nhead=8):
        super(TransformerEncoder, self).__init__()
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)
        
    def forward(self, inputs):
        
        inputs = inputs.transpose(0,1)
        output = self.transformer_encoder(inputs)
        output = output.transpose(0,1)
        
        return output

In [22]:

# アコースティックモデル
class AcousticModelCRnn8Dropout(nn.Module):
    def __init__(self, classes_num, midfeat, momentum):
        super(AcousticModelCRnn8Dropout, self).__init__()

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=48, momentum=momentum)
        self.conv_block2 = ConvBlock(in_channels=48, out_channels=64, momentum=momentum)
        self.conv_block3 = ConvBlock(in_channels=64, out_channels=96, momentum=momentum)
        self.conv_block4 = ConvBlock(in_channels=96, out_channels=128, momentum=momentum)

        self.fc5 = nn.Linear(midfeat, 768, bias=False)
        self.bn5 = nn.BatchNorm1d(768, momentum=momentum)
        
        self.ln1 = nn.LayerNorm(768)
        self.pe = PositionalEncoding(d_model=768, dropout=0.1)

        self.fc = nn.Linear(768, classes_num, bias=True)
        
        self.encoder_layer = TransformerEncoder()
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.fc5)
        init_bn(self.bn5)
        init_layer(self.fc)
        init_ln(self.ln1)

    def forward(self, input):
        """
        Args:
          input: (batch_size, channels_num, time_steps, freq_bins)
        Outputs:
          output: (batch_size, time_steps, classes_num)
        """

        x = self.conv_block1(input, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training) # [batch_size, 128, 1001, 14]
        
        x = x.transpose(1, 2).flatten(2)  # [batch_size, 1001, 1792]

        x = F.relu(self.fc5(x)) # [batch_size, 1001, 768]
        x = self.ln1(x)
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)  # [batch_size, 1001, 768]
        x = self.pe(x) # [batch_soze, 1001, 768]
        
        x = self.encoder_layer(x)  # [batch_size, 1001, 768]
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        
        output = torch.sigmoid(self.fc(x))
        
        # この下にTransformerのEncoder部分(4層)を書く(to do)
        
        return output

In [28]:
class AcousticModelTransformer(nn.Module):
    def __init__(self, classes_num, midfeat, momentum):
        super(AcousticModelTransformer, self).__init__()

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=48, momentum=momentum)
        self.conv_block2 = ConvBlock(in_channels=48, out_channels=64, momentum=momentum)
        self.conv_block3 = ConvBlock(in_channels=64, out_channels=96, momentum=momentum)
        self.conv_block4 = ConvBlock(in_channels=96, out_channels=128, momentum=momentum)

        self.fc5 = nn.Linear(midfeat, 768, bias=False)
        self.bn5 = nn.BatchNorm1d(768, momentum=momentum)
        
        self.ln1 = nn.LayerNorm(768)
        self.pe = PositionalEncoding(d_model=768, dropout=0.1)

        self.fc = nn.Linear(768, classes_num, bias=True)
        
        self.encoder_layer = TransformerEncoder()
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.fc5)
        init_bn(self.bn5)
        init_layer(self.fc)
        init_ln(self.ln1)

    def forward(self, input):
        """
        Args:
          input: (batch_size, channels_num, time_steps, freq_bins)
        Outputs:
          output: (batch_size, time_steps, classes_num)
        """

        x = self.conv_block1(input, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training) # [batch_size, 128, 1001, 14]
        
        x = x.transpose(1, 2).flatten(2)  # [batch_size, 1001, 1792]

        x = F.relu(self.fc5(x)) # [batch_size, 1001, 768]
        x = self.ln1(x)
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)  # [batch_size, 1001, 768]
        x = self.pe(x) # [batch_soze, 1001, 768]
        
        x = self.encoder_layer(x)  # [batch_size, 1001, 768]
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        
        output = torch.sigmoid(self.fc(x))
        
        # この下にTransformerのEncoder部分(4層)を書く(to do)
        
        return output


In [30]:

class Regress_onset_offset_frame_velocity_CRNN(nn.Module):
    def __init__(self, frames_per_second, classes_num):
        super(Regress_onset_offset_frame_velocity_CRNN, self).__init__()
        
        sample_rate = 16000
        window_size = 2048
        hop_size = sample_rate // frames_per_second
        mel_bins = 229
        fmin = 30
        fmax = sample_rate // 2
        
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        
        midfeat = 1792
        momentum = 0.01
        
        # Spectrogram
        self.spectrogram_extractor = Spectrogram(n_fft=window_size,
            hop_length=hop_size, win_length=window_size, window=window,
            center=center, pad_mode=pad_mode, freeze_parameters=True)
        
        # Logmel
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate,
            n_fft=window_size, n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref,
            amin=amin, top_db=top_db, freeze_parameters=True)
        
        self.bn0 = nn.BatchNorm2d(mel_bins, momentum)
        
        self.frame_model = AcousticModelCRnn8Dropout(classes_num, midfeat, momentum)
        self.reg_onset_model = AcousticModelCRnn8Dropout(classes_num, midfeat, momentum)
        self.reg_offset_model = AcousticModelCRnn8Dropout(classes_num, midfeat, momentum)
        self.velocity_model = AcousticModelTransformer(classes_num, midfeat, momentum)
        
        # after CRNN block
        # only onset and frame is required
        # "attention": figs (high_resolution and exploring transformer's pottential) is different but same model expect velocity network
        self.reg_onset_gru = nn.GRU(input_size=88 * 2, hidden_size=256, num_layers=1,
            bias=True, batch_first=True, dropout=0., bidirectional=True)
        self.reg_onset_fc = nn.Linear(512, classes_num, bias=True)
        
        self.frame_gru = nn.GRU(input_size=88 * 3, hidden_size=256, num_layers=1,
            bias=True, batch_first=True, dropout=0., bidirectional=True)
        self.frame_fc = nn.Linear(512, classes_num, bias=True)
        
        self.init_weight()
        
    def init_weight(self):
        init_bn(self.bn0)
        init_gru(self.reg_onset_gru)
        init_gru(self.frame_gru)
        init_layer(self.reg_onset_fc)
        init_layer(self.frame_fc)
        
    def forward(self, input):
        """
        Args:
            input: (batch_size, data_length)
        
        Outputs:
            output_dict: dict, {
              'reg_onset_output': (batch_size, time_steps, classes_num),
              'reg_offset_output': (batch_size, time_steps, classes_num),
              'frame_output': (batch_size, time_steps, classes_num),
              'velocity_output': (batch_size, time_steps, classes_num)
            }
        """
        
        x = self.spectrogram_extractor(input) # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x) # (batch_size, 1, time_step, freq_bins)
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        frame_output = self.frame_model(x)
        reg_onset_output = self.reg_onset_model(x)
        reg_offset_output = self.reg_offset_model(x)
        velocity_output = self.velocity_model(x)
        
        # Concatenete veloacity and onset output to regress final onset output
        x = torch.cat((reg_onset_output, (reg_onset_output ** 0.5) * velocity_output.detach()), dim=2)
        (x, _) = self.reg_onset_gru(x)
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        reg_onset_output = torch.sigmoid(self.reg_onset_fc(x))
        """(batch_size, time_steps, classes_num)"""
        
        # concatenate on/offset and frame outputs to classifier final pitch output
        x = torch.cat((frame_output, reg_onset_output.detach(), reg_offset_output.detach()), dim=2)
        (x, _) = self.frame_gru(x)
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        frame_output = torch.sigmoid(self.frame_fc(x)) # (batch_size,  time_steps, classes_num)
        
        output_dict = {
            'reg_onset_output': reg_onset_output,
            'reg_offset_output': reg_offset_output,
            'frame_output': frame_output,
            'velocity_output': velocity_output}
        
        return output_dict


In [36]:
inputs_batch = torch.ones(10, 2000)
NET = Regress_onset_offset_frame_velocity_CRNN(frames_per_second=100, classes_num = 88)
outputs = NET(inputs_batch)

In [38]:
outputs

{'reg_onset_output': tensor([[[0.4472, 0.5248, 0.4360,  ..., 0.4015, 0.3873, 0.5748],
          [0.3412, 0.5455, 0.5361,  ..., 0.3126, 0.4778, 0.4953],
          [0.5900, 0.4921, 0.5005,  ..., 0.3069, 0.6806, 0.5425],
          ...,
          [0.6006, 0.6217, 0.5187,  ..., 0.3102, 0.3061, 0.4669],
          [0.4405, 0.5557, 0.6065,  ..., 0.4622, 0.6525, 0.5339],
          [0.6069, 0.6280, 0.5871,  ..., 0.6288, 0.5753, 0.3968]],
 
         [[0.4576, 0.5362, 0.4016,  ..., 0.3953, 0.4700, 0.4444],
          [0.5117, 0.4825, 0.4133,  ..., 0.3660, 0.5814, 0.5054],
          [0.4855, 0.5966, 0.4856,  ..., 0.4008, 0.6052, 0.5014],
          ...,
          [0.5427, 0.3981, 0.3756,  ..., 0.4476, 0.5081, 0.3226],
          [0.3589, 0.6175, 0.6390,  ..., 0.3305, 0.4635, 0.2567],
          [0.4383, 0.5579, 0.4697,  ..., 0.5143, 0.5926, 0.4080]],
 
         [[0.4521, 0.5502, 0.4157,  ..., 0.4810, 0.5748, 0.3728],
          [0.5716, 0.5546, 0.4889,  ..., 0.3648, 0.6444, 0.2292],
          [0.4312, 0